# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 3: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 24.10.25 23:59__ \
__Жесткий дедлайн 26.10.25 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 14. То есть за все дз можно получить 22 балла (но не радуйтесь рано, это не так просто). Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [112]:
# !pip install -U datasets==3.4.0

In [21]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003")

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [22]:
dataset['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [23]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [ ]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


In [ ]:
example["ner_tags"]

[0, 5, 0, 5, 0, 1, 0, 0, 0]

In [ ]:
tokenized_text.word_ids()

[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, None]

In [ ]:
tokenized_text.word_ids(batch_index=0)

[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, None]

__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [ ]:
aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

Выровненные метки: [-100    0    5    0    5    0    1    2    2    0    0    0    0 -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


In [24]:
def align_labels_with_tokens(ner_tags, tokenized_input):
    word_ids = tokenized_input.word_ids(batch_index=0)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(ner_tags[word_idx])
        else:
            prev_label = ner_tags[word_idx]
            if prev_label % 2 == 1:
                label_ids.append(prev_label + 1)
            else:
                label_ids.append(prev_label)
        previous_word_idx = word_idx

    return label_ids


In [25]:
def tokenize_and_align(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True,
                                max_length=128, padding="max_length")
    aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_inputs)

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

example = dataset["train"][12]
aligned_labels = align_labels_with_tokens(example["ner_tags"],
                                         tokenizer(example["tokens"], is_split_into_words=True))
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия:", tags)

tokenized_dataset = dataset.map(tokenize_and_align, batched=False, remove_columns=dataset["train"].column_names)

Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]
Выровненные названия: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Отчет: ну тут все понятно, написал функцию align_labels_with_tokens и токенизировал весь датасет

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [7]:
# ! pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2cf990e98224195818ddc7e9feba441d07aab0f4f4f3c887d7e42b08d020269d
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [16]:
from seqeval.metrics import f1_score

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForTokenClassification LOAD REPORT from: bert-base-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized beca

Число параметров: 107726601


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.226013
1000,0.070977
1500,0.046242
2000,0.030005
2500,0.024485


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=2634, training_loss=0.0765915833917095, metrics={'train_runtime': 989.0671, 'train_samples_per_second': 42.589, 'train_steps_per_second': 2.663, 'total_flos': 2751824963545344.0, 'train_loss': 0.0765915833917095, 'epoch': 3.0})

In [69]:
from seqeval.metrics import f1_score
import numpy as np

id2label = {i: label for i, label in enumerate(label_names)}

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    true_predictions = []

    for pred_seq, label_seq in zip(predictions, labels):
        cur_true = []
        cur_pred = []

        for p_i, l_i in zip(pred_seq, label_seq):
            if l_i == -100:
                continue
            cur_true.append(id2label[l_i])
            cur_pred.append(id2label[p_i])

        true_labels.append(cur_true)
        true_predictions.append(cur_pred)

    return {"f1": f1_score(true_labels, true_predictions)}

In [ ]:

preds_logits, true_labels, _ = trainer.predict(tokenized_dataset["test"])

metrics = compute_metrics((preds_logits, true_labels))
print("Test F1:", metrics["f1"])

Test F1: 0.900853807283499


In [ ]:
model.save_pretrained("/content/drive/MyDrive/ner_teacher_model")
tokenizer.save_pretrained("/content/drive/MyDrive/ner_teacher_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('/content/drive/MyDrive/ner_teacher_model/tokenizer_config.json',
 '/content/drive/MyDrive/ner_teacher_model/tokenizer.json')

отчет: скачал модель, сделал файнтюн на 3 эпохах и получил F1 0.9

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [ ]:
import torch
import torch.nn as nn

class FactorizedEmbedding(nn.Module):
    def __init__(self, original_embedding: nn.Embedding, rank: int = 64):
        super().__init__()
        self.vocab_size, self.hidden_size = original_embedding.weight.shape
        self.rank = rank

        self.W1 = nn.Parameter(torch.empty(self.vocab_size, rank))
        self.W2 = nn.Parameter(torch.empty(rank, self.hidden_size))

        with torch.no_grad():
            U, S, Vh = torch.linalg.svd(original_embedding.weight, full_matrices=False)
            self.W1.data = U[:, :rank] @ torch.diag(torch.sqrt(S[:rank]))
            self.W2.data = torch.diag(torch.sqrt(S[:rank])) @ Vh[:rank, :]

    def forward(self, input_ids):
        return torch.matmul(self.W1[input_ids], self.W2)


In [ ]:
#model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_teacher_model")

original_embeddings = model.bert.embeddings.word_embeddings
model.bert.embeddings.word_embeddings = FactorizedEmbedding(original_embeddings, rank=64)

print(model.bert.embeddings.word_embeddings)

FactorizedEmbedding()


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print("Число параметров:", total_params)

Число параметров: 87362569


In [ ]:
training_args = TrainingArguments(
    output_dir="./tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss
500,0.022440
1000,0.021218
1500,0.016082
2000,0.011775
2500,0.011256
3000,0.011294
3500,0.008326


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=3512, training_loss=0.01463106546152154, metrics={'train_runtime': 1391.8277, 'train_samples_per_second': 40.353, 'train_steps_per_second': 2.523, 'total_flos': 3751265644022784.0, 'train_loss': 0.01463106546152154, 'epoch': 4.0})

In [ ]:
preds_logits, true_labels, _ = trainer.predict(tokenized_dataset["test"])

metrics = compute_metrics((preds_logits, true_labels))
print("Test F1:", metrics["f1"])

Test F1: 0.8767888307155323


отчет: для уменьшения числа параметров эмбеддингов создан класс FactorizedEmbedding, разложивший исходную матрицу на две меньшие (ранг=64, инициализация через SVD). После замены слоя эмбеддингов и дообучения модели на удалось сократить параметры примерно на 20М и получить F1 0.87 на тестовой выборке

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="800">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
teacher_model = AutoModelForTokenClassification.from_pretrained(
    "/content/drive/MyDrive/ner_teacher_model"
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/ner_teacher_model"
)

teacher_model.eval()

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from transformers import AutoModelForTokenClassification, BertConfig

teacher_config = teacher_model.config

student_config = BertConfig(
    vocab_size=teacher_config.vocab_size,
    hidden_size=384,
    num_hidden_layers=6,
    num_attention_heads=6,
    intermediate_size=768,
    num_labels=teacher_config.num_labels
)

student_model = AutoModelForTokenClassification.from_config(student_config)

In [ ]:
total_params = sum(p.numel() for p in student_model.parameters())
print("Число параметров ученик:", total_params)

Число параметров ученик: 18439305


In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification


ce_loss = nn.CrossEntropyLoss(ignore_index=-100)
kl_loss = nn.KLDivLoss(reduction="batchmean")

T = 2.0
alpha = 0.5

data_collator = DataCollatorForTokenClassification(tokenizer)

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=16,
    shuffle=True,
    collate_fn=data_collator
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)
student_model.to(device)
student_model.train()
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)

num_epochs = 23
for epoch in range(num_epochs):
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids,
                                           attention_mask=attention_mask).logits

        student_logits = student_model(input_ids=input_ids,
                                       attention_mask=attention_mask).logits

        loss_hard = ce_loss(student_logits.view(-1, student_logits.size(-1)),
                            labels.view(-1))

        student_log_probs = F.log_softmax(student_logits / T, dim=-1)
        teacher_probs = F.softmax(teacher_logits / T, dim=-1)

        mask = labels != -100
        student_log_probs = student_log_probs[mask]
        teacher_probs = teacher_probs[mask]

        loss_soft = kl_loss(student_log_probs, teacher_probs) * (T**2)

        loss = alpha * loss_soft + (1 - alpha) * loss_hard

        loss.backward()
        optimizer.step()

        global_step = epoch * len(train_loader) + step
        if global_step % 100 == 0:
            print(f"Step {global_step} | Loss: {loss.item():.4f}")

Step 0 | Loss: 1.0329
Step 100 | Loss: 0.8027
Step 200 | Loss: 0.8111
Step 300 | Loss: 0.7817
Step 400 | Loss: 0.9125
Step 500 | Loss: 0.8698
Step 600 | Loss: 0.4519
Step 700 | Loss: 0.7746
Step 800 | Loss: 1.1808
Step 900 | Loss: 0.4619
Step 1000 | Loss: 0.5805
Step 1100 | Loss: 0.6883
Step 1200 | Loss: 0.5049
Step 1300 | Loss: 0.5322
Step 1400 | Loss: 0.4688
Step 1500 | Loss: 0.5663
Step 1600 | Loss: 0.3339
Step 1700 | Loss: 0.5130
Step 1800 | Loss: 0.1630
Step 1900 | Loss: 0.7422
Step 2000 | Loss: 0.1934
Step 2100 | Loss: 0.2211
Step 2200 | Loss: 0.2202
Step 2300 | Loss: 0.4970
Step 2400 | Loss: 0.2304
Step 2500 | Loss: 0.2489
Step 2600 | Loss: 0.2975
Step 2700 | Loss: 0.1120
Step 2800 | Loss: 0.4083
Step 2900 | Loss: 0.4632
Step 3000 | Loss: 0.0496
Step 3100 | Loss: 0.1903
Step 3200 | Loss: 0.1861
Step 3300 | Loss: 0.1439
Step 3400 | Loss: 0.3743
Step 3500 | Loss: 0.2464
Step 3600 | Loss: 0.1855
Step 3700 | Loss: 0.1325
Step 3800 | Loss: 0.2281
Step 3900 | Loss: 0.1500
Step 4000 | 

In [ ]:
from transformers import get_linear_schedule_with_warmup

alpha = 0.6
T = 3.0
lr = 2e-5
batch_size = 16
num_epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
student_model.to(device)
student_model.train()


train_loader = DataLoader(tokenized_dataset["train"], batch_size=batch_size, shuffle=True,collate_fn=data_collator)
val_loader   = DataLoader(tokenized_dataset["validation"], batch_size=batch_size,collate_fn=data_collator)

optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
num_training_steps = num_epochs * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=num_warmup_steps,
                                            num_training_steps=num_training_steps)


for epoch in range(num_epochs):
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits

        student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits

        loss_hard = ce_loss(student_logits.view(-1, student_logits.size(-1)),
                            labels.view(-1))

        student_log_probs = F.log_softmax(student_logits / T, dim=-1)
        teacher_probs = F.softmax(teacher_logits / T, dim=-1)
        mask = labels != -100
        student_log_probs = student_log_probs[mask]
        teacher_probs = teacher_probs[mask]
        loss_soft = kl_loss(student_log_probs, teacher_probs) * (T**2)

        loss = alpha * loss_soft + (1 - alpha) * loss_hard
        loss.backward()
        optimizer.step()
        scheduler.step()

        if step % 500 == 0:
            print(f"Epoch {epoch} | Step {step} | Loss: {loss.item():.4f}")


Epoch 0 | Step 0 | Loss: 0.0231
Epoch 0 | Step 500 | Loss: 0.0165
Epoch 1 | Step 0 | Loss: 0.0722
Epoch 1 | Step 500 | Loss: 0.0513


In [ ]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForTokenClassification(tokenizer)

test_loader = DataLoader(tokenized_dataset["test"], batch_size=16, collate_fn=data_collator)

student_model.eval()
preds_logits = []
labels_list = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
        preds_logits.append(logits.cpu())
        labels_list.append(labels.cpu())

preds_logits = torch.cat(preds_logits, dim=0).numpy()
labels_list = torch.cat(labels_list, dim=0).numpy()

metrics = compute_metrics((preds_logits, labels_list))
print("Student Test F1:", metrics["f1"])

Student Test F1: 0.7013340130852239


Student модель создана на основе конфигурации teacher модели с 4 слоями, hidden_size=384, 6 голов внимания и вышло 18,5М параметров. Он обучался на предсказаниях дообученного teacher с комбинированной функцией потерь: кросс-энтропия + KL-дивергенци. Пришлось несколько раз дообучать, в первый раз забыл про warmup. В итоге получилось добиться F1 0.7

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничения. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (1 балл).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (1 балл).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.

## Шеринг весов

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_teacher_model")

original_embeddings = model.bert.embeddings.word_embeddings
model.bert.embeddings.word_embeddings = FactorizedEmbedding(original_embeddings, rank=256)

shared_layer = model.bert.encoder.layer[0]
for i in range(len(model.bert.encoder.layer)):
    model.bert.encoder.layer[i] = shared_layer

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print("Число параметров:", total_params)

Число параметров: 15110665


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss
500,0.258462
1000,0.202800
1500,0.153052
2000,0.126002
2500,0.106764


TrainOutput(global_step=2634, training_loss=0.16603339614520585, metrics={'train_runtime': 898.2541, 'train_samples_per_second': 46.894, 'train_steps_per_second': 2.932, 'total_flos': 476066613733632.0, 'train_loss': 0.16603339614520585, 'epoch': 3.0})

In [ ]:
preds_logits, true_labels, _ = trainer.predict(tokenized_dataset["test"])

metrics = compute_metrics((preds_logits, true_labels))
print("Test F1:", metrics["f1"])

Test F1: 0.7164691458929301


Отчет: ну тут все понятно, сделал факторизацию и шеринг весов, количество параметров 15кк, F1 0.72. что чуть больше чем при дистиляции получилось, хотя параметров меньше

## Факторизация промежуточных слоев

In [59]:
import torch
import torch.nn as nn

class FactorizedLinear(nn.Module):

    def __init__(self, original_linear: nn.Linear, rank: int):
        super().__init__()
        self.in_features = original_linear.in_features
        self.out_features = original_linear.out_features
        self.rank = min(rank, self.in_features, self.out_features)

        self.fc1 = nn.Linear(self.in_features, self.rank, bias=False)
        self.fc2 = nn.Linear(self.rank, self.out_features, bias=original_linear.bias is not None)

        with torch.no_grad():
            W = original_linear.weight.data
            U, S, Vh = torch.linalg.svd(W, full_matrices=False)

            U_r = U[:, :self.rank]
            S_r = S[:self.rank]
            Vh_r = Vh[:self.rank, :]

            self.fc1.weight.data = torch.diag(torch.sqrt(S_r)) @ Vh_r
            self.fc2.weight.data = U_r @ torch.diag(torch.sqrt(S_r))

            if original_linear.bias is not None:
                self.fc2.bias.data = original_linear.bias.data.clone()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [67]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_teacher_model")
model.to(device)

rank=64

for layer in model.bert.encoder.layer:
    layer.attention.self.query = FactorizedLinear(layer.attention.self.query, rank)
    layer.attention.self.key = FactorizedLinear(layer.attention.self.key, rank)
    layer.attention.self.value = FactorizedLinear(layer.attention.self.value, rank)
    layer.attention.output.dense = FactorizedLinear(layer.attention.output.dense, rank)

    layer.intermediate.dense = FactorizedLinear(layer.intermediate.dense, rank)
    layer.output.dense = FactorizedLinear(layer.output.dense, rank)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

In [72]:
print('Количество параметров:', sum(p.numel() for p in model.parameters()))

Количество параметров: 33408777


In [73]:
training_args = TrainingArguments(
    output_dir="./tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)
trainer.train()

Step,Training Loss
500,0.487984
1000,0.215702
1500,0.166671
2000,0.134852
2500,0.120857


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=2634, training_loss=0.21988963718747404, metrics={'train_runtime': 376.0196, 'train_samples_per_second': 112.023, 'train_steps_per_second': 7.005, 'total_flos': 347608873675008.0, 'train_loss': 0.21988963718747404, 'epoch': 3.0})

In [74]:
preds_logits, true_labels, _ = trainer.predict(tokenized_dataset["test"])

metrics = compute_metrics((preds_logits, true_labels))
print("Test F1:", metrics["f1"])

Test F1: 0.7728534185518461


Отчет: Для каждого линейного слоя (включая слои внимания и feed-forward) исходная матрица весов заменялась двумя последовательными слоями меньшего ранга, что позволяет существенно сократить число параметров. Инициализация производилась с помощью SVD-разложения исходных весов, что обеспечивает хорошее начальное приближение и ускоряет последующее дообучение. Ранг был выбран равным 64, что позволило снизить общее количество параметров модели с 108 млн до примерно 33 млн. После замены всех слоёв модель была дообучена в течение нескольких эпох. На тестовом наборе значение F1 составило около 0.77

## Прунинг

In [108]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/ner_teacher_model")
model.to(device)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [109]:
from sparse_gpt import sparse_gpt_layer

data_collator = DataCollatorForTokenClassification(tokenizer)

calibration_loader = torch.utils.data.DataLoader(
    tokenized_dataset["train"].select(range(128)),
    batch_size=4,
    collate_fn=data_collator
)

layer_inputs = {}
handles = []
def hook_fn(name):
    def hook(module, inp, out):
        layer_inputs[name] = inp[0].detach().cpu()
    return hook

for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        handles.append(module.register_forward_hook(hook_fn(name)))

model.eval()
with torch.no_grad():
    for batch in calibration_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        model(**batch)

for h in handles:
    h.remove()

sparsity = 0.7
for name, module in model.named_modules():
    if isinstance(module, nn.Linear) and name in layer_inputs:
        print(f"Обработка {name}")
        inp = layer_inputs[name].to(device)
        inp_2d = inp.view(-1, inp.shape[-1])
        print(f"Обработка {name}, форма весов {module.weight.shape}")
        new_weight = sparse_gpt_layer(module.weight.data, inp_2d, sparsity)
        print(f"Доля нулей до: {(module.weight == 0).float().mean().item():.4f}")
        module.weight.data = new_weight
        print(f"Доля нулей после: {(module.weight == 0).float().mean().item():.4f}")

Обработка bert.encoder.layer.0.attention.self.query
Обработка bert.encoder.layer.0.attention.self.query, форма весов torch.Size([768, 768])
Доля нулей до: 0.0000
Доля нулей после: 0.6992
Обработка bert.encoder.layer.0.attention.self.key
Обработка bert.encoder.layer.0.attention.self.key, форма весов torch.Size([768, 768])
Доля нулей до: 0.0000
Доля нулей после: 0.6992
Обработка bert.encoder.layer.0.attention.self.value
Обработка bert.encoder.layer.0.attention.self.value, форма весов torch.Size([768, 768])
Доля нулей до: 0.0000
Доля нулей после: 0.6992
Обработка bert.encoder.layer.0.attention.output.dense
Обработка bert.encoder.layer.0.attention.output.dense, форма весов torch.Size([768, 768])
Доля нулей до: 0.0000
Доля нулей после: 0.6992
Обработка bert.encoder.layer.0.intermediate.dense
Обработка bert.encoder.layer.0.intermediate.dense, форма весов torch.Size([3072, 768])
Доля нулей до: 0.0000
Доля нулей после: 0.6992
Обработка bert.encoder.layer.0.output.dense
Обработка bert.encoder.l

In [110]:
def count_nonzero_params(model):
    nonzero = 0
    for param in model.parameters():
        nonzero += torch.count_nonzero(param).item()
    return nonzero

total_params = sum(p.numel() for p in model.parameters())
active_params = count_nonzero_params(model)

print(f"\nВсего параметров: {total_params / 1e6:.2f}M")
print(f"Активных параметров: {active_params / 1e6:.2f}M")


Всего параметров: 107.73M
Активных параметров: 48.32M


In [111]:
training_args = TrainingArguments(
    output_dir="./tmp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
)
preds_logits, true_labels, _ = trainer.predict(tokenized_dataset["test"])

metrics = compute_metrics((preds_logits, true_labels))
print("Test F1:", metrics["f1"])

Test F1: 0.6467725645028005


Отчет: был реализован метод одноразового прунинга SparseGPT. Основная идея метода заключается в удалении части весов каждого линейного слоя с последующей корректировкой оставшихся таким образом, чтобы выход слоя изменился минимально.  Изначально я ознакомился с оригинальной статьёй и авторской реализацией на GitHub, но, потом, увидев, что нельзя использовать код даже оставив ссылку сделал упрощенный вариант реализации на основе того как понял этот метод. Модифицированная версия работает следующим образом: для каждого слоя собираются входные активации на небольшом наборе калибровочных примеров (128), по ним строится матрица Гессе  с демпфированием диагонали, затем вычисляется обратная матрица. Для каждого выходного нейрона  веса сортируются по важности, наименее важные обнуляются, и сразу вычисляется суммарная поправка от всех удалённых весов, которая добавляется к оставшимся. В итоге сократил параметры до 48М и получил метрику 0.65, что довольно мало, учитывая что даже не 20М параметров, то есть прунинг слишком жесток

## Общий отчет

В рамках работы решалась задача распознавания именованных сущностей на датасете CoNLL‑2003 с использованием модели BERT‑base. После стандартного дообучения модель достигла F1 = 0.90 при 107 миллионах параметров. Далее были последовательно опробованы различные методы сжатия для уменьшения размера модели с минимальной потерей качества.

Сначала была выполнена факторизация матрицы эмбеддингов с рангом 64, что позволило сократить число параметров примерно на 20 миллионов (до ~87 млн) и после дообучения получить F1 = 0.87.

Затем была реализована дистилляция знаний: модель‑ученик с 4 слоями, скрытой размерностью 384 и 6 головами внимания (18.5 млн параметров) обучалась на предсказаниях учителя с использованием комбинации кросс‑энтропии и KL‑дивергенции, в результате удалось достичь F1 = 0.70.

Далее была применена идея шеринга весов между слоями в сочетании с уже выполненной факторизацией эмбеддингов, что дало модель всего с 15 миллионами параметров и F1 = 0.72, что оказалось лучше дистилляции при ещё меньшем размере.

После этого была проведена факторизация всех линейных слоёв (внимания и feed‑forward) с тем же рангом 64, что сократило параметры до 33 миллионов и после дообучения позволило получить F1 = 0.77.

Наконец, был испытан метод одноразового прунинга SparseGPT: на калибровочных данных вычислялась матрица Гессе для каждого слоя, веса сортировались по важности, наименее важные обнулялись, а оставшиеся корректировались для сохранения выхода слоя. При удалении 70% весов модель сократилась до 48 миллионов активных параметров, однако качество упало до F1 = 0.65, что говорит о слишком агрессивном разреживании без последующего дообучения.

Таким образом, наилучший компромисс между размером и качеством продемонстрировали шеринг весов (15 млн, F1 = 0.72) и факторизация всех слоёв (33 млн, F1 = 0.77), тогда как дистилляция и прунинг уступили по точности при сопоставимом или даже большем числе параметров.

